<a href="https://colab.research.google.com/github/ancgate/cis4400-summer-2025/blob/main/pythonscripts/extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob
!pip install pymongo

In [ ]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# read the config file that is json
import json
with open('config.json') as config_file:
    config = json.load(config_file)

# read information about the source
SOURCE_CONNECTION_STRING = config['SOURCE_CONNECTION_STRING']
SOURCE_CONTAINER_NAME = config['SOURCE_CONTAINER_NAME']
SOURCE_FILE_NAME = config['SOURCE_FILE_NAME']

# read information about the destination
DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config['DESTINATION_FILE_NAME']

# read information about mongodb
MONGODB_CONNECTION_STRING = config['MONGODB_CONNECTION_STRING']
MONGO_DB_NAME = config['MONGO_DB_NAME']
MONGO_COLLECTION_NAME = config['MONGO_COLLECTION_NAME']
print(MONGO_COLLECTION_NAME)